# Prerequisites

### Installs You May Need (if not installed already)

Run the following in terminal:
* pip install jupyter-dash

* Ensure you have all 3 data sets in the Data folder as well as the Chicago Population CSV file

In [ ]:
# Import libraries and dependencies
import panel as pn
pn.extension('plotly')
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import hvplot.pandas
from panel.interact import interact
import plotly.express as px
import dash
from dash import dcc
from dash.dependencies import Input, Output
from dash import html
from jupyter_dash import JupyterDash
import os
from pathlib import Path
from dotenv import load_dotenv

# 1) Data Exploration

In [ ]:
# Change folder path depending on where you are placing these files
# import Chicagos data
chicago = pd.read_csv(r'..\Data\Crimes_-_2001_to_Present.csv')
chicago_pop = pd.read_csv(r'..\Data\Chicago Population.csv')

# import San Fran data
san_fran = pd.read_csv(r'..\Data\Police_Department_Incident_Reports__2018_to_Present.csv')

# import philly data
philly = pd.read_csv(r'..\Data\incidents_part1_part2.csv')

In [ ]:
# Get a view of chicago data
chicago.head()

In [ ]:
# View unique columns, and get a tally
chicago.count()

In [ ]:
# View unique values in a column (Year)
chicago.Year.unique()

In [ ]:
# View unique values in a column (Primary Type)
chicago['Primary Type'].unique()

In [ ]:
chicago['Location Description'].unique()

In [ ]:
# View San Fran data
san_fran.head()

In [ ]:
# View unique columns, and get a tally
san_fran.count()

In [ ]:
# View unique values in a column (Year)
san_fran['Incident Category'].unique()

In [ ]:
# View Philly data
philly.head()

In [ ]:
# View unique header and count
philly.count()

In [ ]:
# View unique values in a column (Year)
philly['text_general_code'].unique()

# 2) Data Cleanup, Filtering, and Visualization/Observations

In [ ]:
# Filter and sort data chicago data

# Rename column (to allow dataframe.column formatting
chicago = chicago.rename(columns={"Primary Type": "Primary_Type"})

# Filter for 2015 through today
chicago = chicago.loc[(chicago['Year'] >= 2015) & (chicago['Year'] < 2022)]

# Sort by year and check year for years filtered in above line
chicago = chicago.sort_values(by=['Year'])
chicago.Year.unique()

In [ ]:
# Filter Chicago dataframe for top 10 crime since 2015
chicago_crime = chicago[['Year','Primary_Type']]
chicago_crime.tail()

In [ ]:
# Filter Chicago dataframe for top 10 crime since 2015
chicago_crime = chicago[['Year','Primary_Type']]
chicago_top_ten = chicago_crime['Primary_Type'].value_counts().head(10).reset_index()
chicago_top_ten = chicago_top_ten.rename(columns={"Primary_Type": "Crime_Count","index": "Crime"})
chicago_top_ten.head(10)

In [ ]:
# Filter chicago_crime data frame to get only data of top ten crimes
crimelist = chicago_top_ten['Crime'].tolist() # create a list storing top 10 in a list
boolean_series = chicago_crime.Primary_Type.isin(crimelist) # See if crime in the df, we will reuse crimelist in 
top_10_chicago_df = chicago_crime[boolean_series] # Apply the series
top_10_chicago_df.Primary_Type.unique()

In [ ]:
# Pivot the data, indexed on Year, showing Primary Type, and getting a count
top_10_df  = top_10_chicago_df.pivot_table(index="Year",
columns="Primary_Type",
aggfunc=len,
fill_value=0)
top_10_df

In [ ]:
# Filter Chicago population data for > 2015
chicago_pop = chicago_pop.loc[(chicago_pop['Year'] >= 2015)]
chicago_pop = chicago_pop.drop(['Change %'], axis=1)
chicago_pop = chicago_pop.set_index('Year')
chicago_pop.plot.line(figsize=(20,7),title='Chicago Pop')

In [ ]:
top_10_df.plot.line(figsize=(20,7),title='Top Ten Crime Count')

### Observations from above visuals:
* From this view is the clear drop in thefts beginning early from 2019. 
* Criminal Damages in red, is fairly flat since 2015 in the count of arrests made. 
* Other categories such as narcotics, robbery, or total crimes that are below 30000 are not as pronounced.
* In 2019, population was still increasing, [Chicago’s downtown was witnessing unprecedented growth — both in population and construction activity. Dozens of high-rises were being built at a time, adding thousands of rental units and hotel rooms to the city each year. And despite the downturn caused by the pandemic, Chicago’s Loop and broader central business district is still the fastest growing neighborhood in the city, and the fastest growing downtown in the nation, a report from the Chicago Loop Alliance suggests.](https://rejournals.com/chicagos-downtown-still-fastest-growing-in-country-report-says/#:~:text=In%20terms%20of%20raw%20numbers,growing%20community%20within%20the%20city.)


In [ ]:
# Using hvplot to demonstrate the same plot, but now dynamically filterable
top_10_linechart = top_10_df.hvplot(width=1100, height=400)
top_10_linechart

In [ ]:
# Calculate % Change
top10_pct_change = top_10_df.pct_change()
top10_pct_change

In [ ]:
# Remove 2015 showing NaN with syntax of iloc df.iloc[row_start:row_end , col_start, col_end]
top10_pct_change = top10_pct_change.iloc[1: , :]
top10_pct_change

In [ ]:
#Plot % change using Pandas .plot.line
top10df = top10_pct_change.plot.line(figsize=(20,7),title='Percent Change Top Ten Crime Types')
top10df

In [ ]:
# Same plot with hvplot
top10df = top10_pct_change.hvplot(width=1100, height=400)
top10df

In [ ]:
# Same plot with plotly express
pct_change_chicago_crime = px.line(top10_pct_change, x=top10_pct_change.index, y=crimelist, 
              title='Percent Change Top 10 Crimes')
pct_change_chicago_crime.update_yaxes(title_text='% Change')
pct_change_chicago_crime.show()

### Observations from above visual
Drops in types of crime beginning covid:
* Theft
* Assault
* ‘Other Offenses’
* Battery
* Narcotics

Rises in types of crimes after covid:
* Motor Vehicle Theft
* Robbery
* Burglary

To validate these findings: [ecofact.org notes:] (https://econofact.org/crime-in-the-time-of-covid)
- Home burglaries dropped, while commercial burglaries and car thefts rose. 
- Drug crimes dropped the most — by 65% — at the pandemic onset and fell again during the summer. Drug crimes saw the biggest decline of any category at the onset of the pandemic in almost all cities for which I have data – 65% on average. 

One of the clearest observable anomalies is Narcotics, affected by Covid as noted by [CWBChicago](https://cwbchicago.com/2020/03/arrests-plummet-crime-nosedives-in-early-days-of-covid-19.html), who cover local police data in chicago:

“Interim Police Supt. Charlie Beck yesterday said officers are stopping fewer pedestrians and drivers. And their “hands off” approach is driving CPD arrest numbers to possibly historic lows.”






In [ ]:
# Calculate the correlation between crimes
correlation = top10_pct_change.corr()
correlation

In [ ]:
#Creating a correlation matrix using plotly express and only color those with correlation > .7
corr_map = px.imshow(round(correlation,2),
               text_auto=True,
               aspect='auto',
               color_continuous_scale='magma',
               zmin=0.7,
               zmax=1,
               width=700,
               height=600)

# Set the visibility of both axis to none to hide axis title, change the x axis angle,and show plot
corr_map.update_yaxes(title=None, showticklabels=True).update_xaxes(title=None, showticklabels=True).update_xaxes(tickangle=90)

### Observations from above visual
Types of crime that show a correlation in occurence
* Assault & Theft
* Assault & Battery
* Robbery & Motor Vehicle theft
* Burglary & Deceptive Practice

In [ ]:
# Filter for thefts
df_theft = chicago[(chicago["Primary_Type"]=="THEFT")]
df_theft = df_theft.sort_values(by=['Year'], ascending=True)

# Filter where the porch thief swiped the goods
boolean_series = df_theft['Location Description'].isin(['RESIDENTIAL YARD (FRONT/BACK)','DRIVEWAY - RESIDENTIAL','RESIDENCE PORCH/HALLWAY','RESIDENCE - PORCH / HALLWAY','RESIDENCE - YARD (FRONT / BACK)'])
porch_pirates_df = df_theft[boolean_series]
porch_pirates_df.sort_index().head(10)
df_theft.head()

In [ ]:
# Read the Mapbox API key, note it should be stored in an .env file where the variable is named 'access_token' and stores your key
# for example access_token = pk.whatever_is_your_token_id
load_dotenv()
map_box_api = os.getenv("access_token")

# Set the Mapbox API
px.set_mapbox_access_token(map_box_api)

In [ ]:
# Plot where the thieves struck on map using mapbox, and have an animation frame to show years
map_porch_pirates = px.scatter_mapbox(
    porch_pirates_df,
    lat="Latitude",
    lon="Longitude",
    color="Description",
    animation_frame="Year",
    height = 600,
    width = 900,
    zoom=9
)

# Display the map
map_porch_pirates.show()

In [ ]:
# Filter and Frame San Fran Data

In [ ]:
# Rename Columns
san_fran = san_fran.rename(columns={"Incident Category": "Incident_Category"})
san_fran = san_fran.rename(columns={"Incident Year": "Year"})

# Filter the Data for Years wanted
san_fran = san_fran.loc[(san_fran['Year'] >= 2018) & (san_fran['Year'] < 2022)] 
san_fran_crime = san_fran[['Year','Incident_Category']]

# Count of Incident
san_fran_top_ten = san_fran_crime['Incident_Category'].value_counts().head(10).reset_index()
san_fran_top_ten = san_fran_top_ten.rename(columns={"Incident_Category": "Crime_Count","index": "Crime"})
san_fran_crimelist = san_fran_top_ten['Crime'].tolist()
san_fran_boolean_series = san_fran_crime.Incident_Category.isin(san_fran_crimelist)
top_10_san_fran_df = san_fran_crime[san_fran_boolean_series]

# Pivot data
san_fran_top_10_df  = top_10_san_fran_df.pivot_table(index="Year",
columns="Incident_Category",
aggfunc=len,
fill_value=0)

In [ ]:
san_fran_top_10_df.plot.line(figsize=(20,7),title='Top Ten Crime Count')

In [ ]:
# Plot of the percent change
san_fran_top10_pct_change = san_fran_top_10_df.pct_change()

# Syntax of iloc df.iloc[row_start:row_end , col_start, col_end]
san_fran_top10_pct_change = san_fran_top10_pct_change.iloc[1: , :]
san_fran_top10_pct_change

# Plot top ten Categories in San Fran
san_fran_top10_pct_change.plot.line(figsize=(20,7),title='Percent Change in Top Ten Crimes')

In [ ]:
# Same plot with plotly express
pct_change_chicago_sf = px.line(san_fran_top10_pct_change, x=san_fran_top10_pct_change.index, y=san_fran_crimelist, 
              title='Percent Change Top 10 Crimes')
pct_change_chicago_sf.update_yaxes(title_text='% Change')
pct_change_chicago_sf.show()

In [ ]:
# Calculate the correlation between each column
correlationsf = san_fran_top10_pct_change.corr()
correlationsf

In [ ]:
# Use the `heatmap` function from the Seaborn library to visualize correlations
san_fran_heat_map= px.imshow(round(correlationsf,2),
               text_auto=True,
               aspect='auto',
               color_continuous_scale='magma',
               zmin=0.7,
               zmax=1,
               width=700,
               height=600)

# Set the visibility of both axis to none to hide axis title, change the x axis angle,and show plot
san_fran_heat_map.update_yaxes(title=None, showticklabels=True).update_xaxes(title=None, showticklabels=True).update_xaxes(tickangle=90)
san_fran_heat_map

In [ ]:
# Filter for thefts
df_theft = san_fran[(san_fran["Incident_Category"]=="Larceny Theft") | (san_fran["Incident_Category"]=="Burglary")]
df_theft = df_theft.sort_values(by=['Year'], ascending=True)
boolean_series = df_theft['Incident Subcategory'].isin(['Larceny - Auto Parts','Larceny Theft - Bicycle','Larceny Theft - From Building','Larceny Theft - Other', 'Burglary - Residential'])
san_fran_porch_pirates_df = df_theft[boolean_series]
san_fran_porch_pirates_df.sort_index().head(10)

san_fran_map_porch_pirates = px.scatter_mapbox(
    san_fran_porch_pirates_df,
    lat="Latitude",
    lon="Longitude",
    color="Incident Subcategory",
    animation_frame="Year",
    height = 800,
    width = 900,
    zoom=10
)


# Set the visibility of both axis to none to hide axis title, change the x axis angle,and show plot

san_fran_map_porch_pirates.show()

In [ ]:
# Bring in philly data
philly = philly.rename(columns={"text_general_code": "general_code"})
philly = philly.rename(columns={"dispatch_date": "Year"})
philly = philly.drop(columns={"the_geom"})
philly = philly.drop(columns={"the_geom_webmercator"})
philly=philly[(philly['Year'] >= '2015-01-01') & (philly['Year'] <= '2021-12-31')]
philly_crime = philly[['Year','general_code']]
philly_top_ten = philly_crime['general_code'].value_counts().head(10).reset_index()
philly_top_ten = philly_top_ten.rename(columns={"general_code": "Crime_Count","index": "Crime"})
crimelist_philly = philly_top_ten['Crime'].tolist()
boolean_series_philly = philly_crime.general_code.isin(crimelist_philly)
top_10_philly_df = philly_crime[boolean_series_philly]
top_10_df  = top_10_philly_df.pivot_table(index="Year",
columns="general_code",
aggfunc=len,
fill_value=0)

In [ ]:
top_10_df.plot.line(figsize=(20,7),title='Top Ten Crime Count')

In [ ]:
# plot of % change
top10_pct_change = top_10_df.pct_change()
top10_pct_change = top10_pct_change.iloc[1: , :]

# Plot Perecent Change Top 10 Crimes
pct_change_chicago_philly = px.line(top10_pct_change, x=top10_pct_change.index, y=crimelist_philly, 
              title='Percent Change Top 10 Crimes')
pct_change_chicago_philly.update_yaxes(title_text='% Change')
pct_change_chicago_philly.show()

In [ ]:
# correlation between each column
correlation_philly = top10_pct_change.corr()
correlation_philly

In [ ]:
philly_heat_map = px.imshow(round(correlation_philly,2),
               text_auto=True,
               aspect='auto',
               color_continuous_scale='magma',
               zmin=0.1,
               zmax=0.8,
               width=700,
               height=600)

# Set the visibility of both axis to none to hide axis title, change the x axis angle,and show plot
philly_heat_map.update_yaxes(title=None, showticklabels=True).update_xaxes(title=None, showticklabels=True).update_xaxes(tickangle=90)
philly_heat_map

In [ ]:
# Filter for Thefts in Philadelphia and display 
df_theft = philly[(philly["general_code"]=="Thefts")]
df_theft = df_theft.sort_values(by=['Year'], ascending=True)
porch_pirates_philly = df_theft[boolean_series_philly]
porch_pirates_philly.sort_index()

# Create Map of Philadelphia thefts
map_porch_pirates_philly = px.scatter_mapbox(
    porch_pirates_philly,
    lat="lat",
    lon="lng",
    color="general_code",
    animation_frame="Year",
    height = 800,
    width = 900,
    zoom=9
)
map_porch_pirates_philly.show()

# 3) Visual Aggregation

In [ ]:
# Create panel tab to display
tabs = pn.Tabs(
  ("Correlations of Crimes in Chicago", corr_map),
  ("Map of Thefts off the 'Porch'", map_porch_pirates),  # Note, try running animation frame within Panel, the play button will not render, nothing appeaers to update! 1 Solution in Dash shown below.
  ("Percent change YoY", pct_change_chicago_crime))
tabs

### One workaround to the above solution for the animation frame not rendering is to use [Dash](https://dash.plotly.com/).

In [ ]:
# Set the dash instance
app = JupyterDash(__name__)

# Create the layout of the html

app.layout = html.Div([
   #Create the dropdown division to select values of one of the 3 cities
   html.Div([dcc.Dropdown(id='dropdown1',options=[{'label': 'Chicago', 'value': 'chic'},{'label': 'San Francisco','value': 'sf'}, {'label': 'Philadelphia','value': 'philly'}],value='city',),],style={'width': '90%', 'display': 'inline-block'}),  
    html.Div(
        [html.Div([html.H3('Map of Porch Piracy'),dcc.Graph(id='g1', figure=map_porch_pirates)]),
         html.Div([html.H3('Correlation of Top Ten Crimes'),dcc.Graph(id='g2', figure=corr_map)]),
         html.Div([html.H3('Percent Change YoY'),dcc.Graph(id='g3', figure=pct_change_chicago_crime)]),
        ])
])

#return which graph plotted previously for one of the 3 cities depending on the value selected from the dropdown

@app.callback(Output('g1', 'figure'),
              Input('dropdown1', 'value'))
def update_graph_a(value):
    if value == 'chic':
        figure_a = map_porch_pirates
    elif value =='sf':
        figure_a = san_fran_map_porch_pirates
    elif value =='philly':
        figure_a = map_porch_pirates_philly
        
    else:figure_a = map_porch_pirates
    return figure_a

@app.callback(Output('g2', 'figure'),
              Input('dropdown1', 'value'))
def update_graph_b(value):
        if value == 'chic':
            figure_b = corr_map
            
        elif value =='sf':
            figure_b = san_fran_heat_map
            
        elif value =='philly':
            figure_b = philly_heat_map
        else: figure_b = corr_map
        return figure_b

@app.callback(Output('g3', 'figure'),
              Input('dropdown1', 'value'))
def update_graph_c(value):
        if value == 'chic':
            figure_c = pct_change_chicago_crime
            
        elif value == 'sf':
            figure_c = pct_change_chicago_sf
        
        elif value =='philly':
            figure_c = pct_change_chicago_philly
        
        else: figure_c = pct_change_chicago_crime    
        
        
        return figure_c


# Run to show
app.run_server(mode="inline")